In [349]:
# imports / libraries used
import pandas as pd
import numpy as np
import ast

### Data Preprocessing (including Holdings and Bond Information)

In [350]:
# load data
etf_data = pd.read_csv("edge_data.csv")

# drop weird index column
etf_data = etf_data.iloc[:, 1:]
etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,holdings_names,holdings_percents,Stock_Data,Var,drawdown,avg_sharpe_ratio,beta,avg_monthly_return,volatility,RSI
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,"['Unilever PLC', 'Shell PLC', 'ASML Holding NV...","[0.1687282, 0.15277919, 0.1046506, 0.091458395...",open ...,-0.012182,-0.224168,-0.943643,0.0,0.016163,0.009947,53.540314
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,"['Unilever PLC', 'Shell PLC', 'ASML Holding NV...","[0.1690629, 0.1530756, 0.104857706, 0.0916288,...",open ...,-0.011809,-0.356065,-0.803378,0.0,0.013677,0.010545,53.611734
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,[],[],open ...,-0.013404,-0.200601,-0.647148,0.0,0.030004,0.013205,65.807441
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,"['Arcadis NV', 'Inpost SA Ordinary Shares', 'W...","[0.10219771, 0.0895253, 0.085002206, 0.0732356...",open ...,-0.013861,-0.423651,-0.784502,0.0,0.005198,0.011083,46.335950
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,"['Erste Group Bank AG.', 'OMV AG', 'Bawag Grou...","[0.21247071, 0.120681696, 0.1080223, 0.0991177...",open ...,-0.014308,-0.484438,-0.656542,0.0,0.010611,0.012936,46.086928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,"['BW LPG Ltd', 'Coface SA', 'Sirius Real Estat...","[0.015789501, 0.011942199, 0.010716501, 0.0102...",open ...,-0.013612,-0.388548,-0.866023,0.0,0.003045,0.010576,43.097363
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,"['BW LPG Ltd', 'Coface SA', 'Sirius Real Estat...","[0.015789501, 0.011942199, 0.010716501, 0.0102...",open ...,-0.012885,-0.383876,-0.866300,0.0,0.007272,0.010376,46.422888
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,"['Microsoft Corp', 'Apple Inc', 'AbbVie Inc', ...","[0.050480798, 0.0376131, 0.0284829, 0.0234956,...",open ...,-0.010444,-0.315128,-0.882943,0.0,0.011188,0.010002,50.094538
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,"['Microsoft Corp', 'Apple Inc', 'AbbVie Inc', ...","[0.050480798, 0.0376131, 0.0284829, 0.0234956,...",open ...,-0.009910,-0.309434,-0.890636,0.0,0.013374,0.009727,50.403902


In [351]:
# extract bond information from Holdings
bond_info = etf_data['Holdings'].apply(eval)
etf_data['bondHoldings'] = bond_info.apply(lambda x: x.get('bondHoldings', {}))
etf_data['bondRatings'] = bond_info.apply(lambda x: x.get('bondRatings', []))

# separate cols bc list of dict, not just dictionary
def separate_bond_ratings(ratings):
    result = {}
    for rating in ratings:
        result.update(rating)
    return result

# create columns for specific attributes of bond ratings
bondRatings_df = pd.json_normalize(etf_data['bondRatings'].apply(separate_bond_ratings))
bondRatings_df.columns = [f'bondRating_{col}' for col in bondRatings_df.columns]
etf_data = pd.concat([etf_data, bondRatings_df], axis=1)

# create columns for specific attributes of bond holdings
bondHoldings_df = pd.json_normalize(etf_data['bondHoldings'])
bondHoldings_df.columns = [f'bondHoldings_{col}' for col in bondHoldings_df.columns]
etf_data = pd.concat([etf_data, bondHoldings_df], axis=1)

etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,bondRating_bb,bondRating_aa,bondRating_aaa,bondRating_a,bondRating_other,bondRating_b,bondRating_bbb,bondRating_below_b,bondHoldings_maturity,bondHoldings_duration
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [352]:
# create columns for each holding and add the percentile as row value under matched holdings

# clean data by converting string representations of lists into real lists
def clean_and_parse(data):
    if data == '[]':
        return []
    return ast.literal_eval(data)

etf_data['holdings_symbols'] = etf_data['holdings_symbols'].apply(clean_and_parse)
etf_data['holdings_names'] = etf_data['holdings_names'].apply(clean_and_parse)
etf_data['holdings_percents'] = etf_data['holdings_percents'].apply(clean_and_parse)

# create dict from symbol, name, and percent columns
def create_holding_dict(symbols, names, percents):
    return [{symbol: {'name': name, 'percent': percent}} for symbol, name, percent in zip(symbols, names, percents)]

etf_data['holding_dict'] = etf_data.apply(lambda row: create_holding_dict(row['holdings_symbols'], row['holdings_names'], row['holdings_percents']), axis=1)

# split dict into symbol columns and percent row values
new_columns = {}
def extract_to_columns(row):
    holding_dict = row['holding_dict']
    for item in holding_dict:
        for symbol, details in item.items():
            # If symbol is not in new_columns, initialize the list with NaN values
            if symbol not in new_columns:
                new_columns[symbol] = [None] * len(etf_data)
            # Update the list with the percent for the current row
            new_columns[symbol][row.name] = details['percent']

etf_data.apply(extract_to_columns, axis=1)
new_df = pd.DataFrame(new_columns)
etf_data = pd.concat([etf_data, new_df], axis=1)

etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,SRE.L,AGR.L,TBCG.L,VATN.SW,OUT1V.HE,ANIM.MI,IF.MI,TEP.L,7267.T,4502.T
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0.010717,0.010219,0.009773,0.009652,0.009598,0.009525,0.008973,0.008713,NaN,NaN
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0.010717,0.010219,0.009773,0.009652,0.009598,0.009525,0.008973,0.008713,NaN,NaN
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [353]:
# drop columns
#       Stock_Data  - general data on fund's holdings, but we have more accesible representation of this in other cols
#       Ticker_Info  - Yfinance object combining ticket and DomicileCountry
#       Holdings  - information covered in other columns (Holdings/Position	Valuation Metrics	Sector Exposure, Bond Holdings/Ratings)
#       Holdings_Symbols - extracted this information to be more usable so not needed anymore
#       Holdings_Names - extracted this information to be more usable so not needed anymore
#       Holdings_Percents - extracted this information to be more usable so not needed anymore
#       bondHoldings - extracted this information to be more usable so not needed anymore
#       bondRatings - extracted this information to be more usable so not needed anymore
#       holding_dict - extracted this information to be more usable so not needed anymore

etf_data = etf_data.drop(columns = ['Stock_Data', 'Ticker_Info', 'Holdings', 'holdings_symbols', 'holdings_names', 'holdings_percents', 'bondRatings', 'bondHoldings', 'holding_dict'])
etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,SRE.L,AGR.L,TBCG.L,VATN.SW,OUT1V.HE,ANIM.MI,IF.MI,TEP.L,7267.T,4502.T
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0.010717,0.010219,0.009773,0.009652,0.009598,0.009525,0.008973,0.008713,NaN,NaN
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0.010717,0.010219,0.009773,0.009652,0.009598,0.009525,0.008973,0.008713,NaN,NaN
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [354]:
# fill zeros for all holding data where we dont have information 
etf_data.iloc[:, -1161:] = etf_data.iloc[:, -1161:].fillna(0)

# where we dont have sector, position, bond data (nan), fill this in as 0 ---> represent percentages so if it's not there, can say 0
fill_zero = ['sector_realestate','sector_consumer_cyclical','sector_basic_materials','sector_consumer_defensive','sector_technology','sector_communication_services','sector_financial_services',
                'sector_utilities','sector_industrials','sector_energy','sector_healthcare','cashPosition','stockPosition','bondPosition','otherPosition','preferredPosition','convertiblePosition',
                'bondRating_us_government','bondRating_bb','bondRating_aa','bondRating_aaa','bondRating_a','bondRating_other','bondRating_b','bondRating_bbb','bondRating_below_b',
                'bondHoldings_maturity','bondHoldings_duration']
for col in fill_zero:
    etf_data[col] = etf_data[col].fillna(0)

# one row doesnt have beta, avg_monthly_return, RSI --> drop this row (Lyxor ETF)
etf_data = etf_data.dropna(subset=['beta'])

# 33 rows dont have var, volatility --> drop these rows (mostly Lyxor ETF)
etf_data = etf_data.dropna(subset=['Var'])

In [355]:
# one-hot encode the labels

if 'labels' in etf_data.columns:
    etf_data['labels'] = etf_data['labels'].apply(clean_and_parse)
else:
    print("The 'labels' column does not exist in the DataFrame. Current columns are:", etf_data.columns)

unique_items = set([item for sublist in etf_data['labels'] for item in sublist])

new_columns = []
for item in unique_items: 
    new_column = etf_data['labels'].apply(lambda x: 1 if item in x else 0)
    new_column.name = item  
    new_columns.append(new_column)

etf_data = pd.concat([etf_data] + new_columns, axis=1)

etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,All-World,Change,Neutral,Equity,Convertible,3-5,Economy,1-3Y,30-15,Bank
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0,0,0,1,0,0,0,0,0,0
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0,0,0,1,0,0,0,0,0,0
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,0,0,0,1,0,0,0,0,0,0
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0,0,0,1,0,0,0,0,0,0
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0,0,0,1,0,0,0,0,0,0
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0,0,0,1,0,0,0,0,0,0
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0,0,0,1,0,0,0,0,0,0
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0,0,0,1,0,0,0,0,0,0


In [356]:
# drop labels column
etf_data = etf_data.drop(columns = 'labels')

In [357]:
# accidentally have 2 a,tech labels - last one comes frm label, first comes from holding tags

def flatten_inner_df(row):
    inner_df = row['A']
    # Flatten inner DataFrame columns to individual columns in the outer DataFrame
    return pd.Series(inner_df.iloc[0])  # Grab the first row, or modify based on needs

# Apply the function to expand the inner DataFrame into columns in the outer DataFrame
expanded_columns = etf_data.apply(flatten_inner_df, axis=1)

# Concatenate the new columns with the original DataFrame
etf_data = pd.concat([etf_data.drop(columns=['A']), expanded_columns], axis=1)
etf_data = etf_data.rename(columns={etf_data.columns[-1]: 'A_1'})


def flatten_inner_df(row):
    inner_df = row['TECH']
    # Flatten inner DataFrame columns to individual columns in the outer DataFrame
    return pd.Series(inner_df.iloc[0])  # Grab the first row, or modify based on needs

# Apply the function to expand the inner DataFrame into columns in the outer DataFrame
expanded_columns = etf_data.apply(flatten_inner_df, axis=1)

# Concatenate the new columns with the original DataFrame
etf_data = pd.concat([etf_data.drop(columns=['TECH']), expanded_columns], axis=1)
etf_data = etf_data.rename(columns={etf_data.columns[-1]: 'Tech_label'})


In [358]:
# confirm data types

str_cols = ['isin', 'wkn', 'name', 'fundProvider', 'legalStructure', 'fundCurrency', 'inceptionDate', 'ticker', 'domicileCountry']

for column in etf_data.columns:
    if column in str_cols:
        etf_data[column] = etf_data[column].astype(str)
    else:
       etf_data[column] = pd.to_numeric(etf_data[column], errors='coerce')


In [359]:
etf_data

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,Neutral,Equity,Convertible,3-5,Economy,1-3Y,30-15,Bank,A_1,Tech_label
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0,1,0,0,0,0,0,0,0.0,0.0
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0,1,0,0,0,0,0,0,0.0,0.0
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,0,1,0,0,0,0,0,0,0.0,0.0
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0,1,0,0,0,0,0,0,0.0,0.0
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0,1,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0,1,0,0,0,0,0,0,0.0,0.0
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0,1,0,0,0,0,0,0,0.0,0.0
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0,1,0,0,0,0,0,0,0.0,0.0
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0,1,0,0,0,0,0,0,0.0,0.0


### Data Preprocessing (without Holding and Bond Information)

In [360]:
# load data
etf_data_small = pd.read_csv("edge_data.csv")

# drop weird index column
etf_data_small = etf_data_small.iloc[:, 1:]
etf_data_small

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,holdings_names,holdings_percents,Stock_Data,Var,drawdown,avg_sharpe_ratio,beta,avg_monthly_return,volatility,RSI
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,"['Unilever PLC', 'Shell PLC', 'ASML Holding NV...","[0.1687282, 0.15277919, 0.1046506, 0.091458395...",open ...,-0.012182,-0.224168,-0.943643,0.0,0.016163,0.009947,53.540314
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,"['Unilever PLC', 'Shell PLC', 'ASML Holding NV...","[0.1690629, 0.1530756, 0.104857706, 0.0916288,...",open ...,-0.011809,-0.356065,-0.803378,0.0,0.013677,0.010545,53.611734
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,[],[],open ...,-0.013404,-0.200601,-0.647148,0.0,0.030004,0.013205,65.807441
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,"['Arcadis NV', 'Inpost SA Ordinary Shares', 'W...","[0.10219771, 0.0895253, 0.085002206, 0.0732356...",open ...,-0.013861,-0.423651,-0.784502,0.0,0.005198,0.011083,46.335950
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,"['Erste Group Bank AG.', 'OMV AG', 'Bawag Grou...","[0.21247071, 0.120681696, 0.1080223, 0.0991177...",open ...,-0.014308,-0.484438,-0.656542,0.0,0.010611,0.012936,46.086928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,"['BW LPG Ltd', 'Coface SA', 'Sirius Real Estat...","[0.015789501, 0.011942199, 0.010716501, 0.0102...",open ...,-0.013612,-0.388548,-0.866023,0.0,0.003045,0.010576,43.097363
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,"['BW LPG Ltd', 'Coface SA', 'Sirius Real Estat...","[0.015789501, 0.011942199, 0.010716501, 0.0102...",open ...,-0.012885,-0.383876,-0.866300,0.0,0.007272,0.010376,46.422888
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,"['Microsoft Corp', 'Apple Inc', 'AbbVie Inc', ...","[0.050480798, 0.0376131, 0.0284829, 0.0234956,...",open ...,-0.010444,-0.315128,-0.882943,0.0,0.011188,0.010002,50.094538
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,"['Microsoft Corp', 'Apple Inc', 'AbbVie Inc', ...","[0.050480798, 0.0376131, 0.0284829, 0.0234956,...",open ...,-0.009910,-0.309434,-0.890636,0.0,0.013374,0.009727,50.403902


In [361]:
# drop columns
#       Stock_Data  - general data on fund's holdings, but we have more accesible representation of this in other cols
#       Ticker_Info  - Yfinance object combining ticket and DomicileCountry
#       Holdings  - information covered in other columns (Holdings/Position	Valuation Metrics	Sector Exposure, Bond Holdings/Ratings)
#       Holdings_Symbols - extracted this information to be more usable so not needed anymore
#       Holdings_Names - extracted this information to be more usable so not needed anymore
#       Holdings_Percents - extracted this information to be more usable so not needed anymore

etf_data_small = etf_data_small.drop(columns = ['Stock_Data', 'Ticker_Info', 'Holdings', 'holdings_symbols', 'holdings_names', 'holdings_percents'])
etf_data_small

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,sector_industrials,sector_energy,sector_healthcare,Var,drawdown,avg_sharpe_ratio,beta,avg_monthly_return,volatility,RSI
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0.1519,0.1534,0.0214,-0.012182,-0.224168,-0.943643,0.0,0.016163,0.009947,53.540314
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0.1520,0.1535,0.0214,-0.011809,-0.356065,-0.803378,0.0,0.013677,0.010545,53.611734
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,NaN,NaN,NaN,-0.013404,-0.200601,-0.647148,0.0,0.030004,0.013205,65.807441
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0.3475,0.1443,0.0511,-0.013861,-0.423651,-0.784502,0.0,0.005198,0.011083,46.335950
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0.1226,0.1313,0.0000,-0.014308,-0.484438,-0.656542,0.0,0.010611,0.012936,46.086928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0.2787,0.0488,0.0344,-0.013612,-0.388548,-0.866023,0.0,0.003045,0.010576,43.097363
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0.2787,0.0488,0.0344,-0.012885,-0.383876,-0.866300,0.0,0.007272,0.010376,46.422888
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0.1596,0.0000,0.2001,-0.010444,-0.315128,-0.882943,0.0,0.011188,0.010002,50.094538
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0.1596,0.0000,0.2001,-0.009910,-0.309434,-0.890636,0.0,0.013374,0.009727,50.403902


In [362]:
# where we dont have sector, position, bond data (nan), fill this in as 0 ---> represent percentages so if it's not there, can say 0
fill_zero = ['sector_realestate','sector_consumer_cyclical','sector_basic_materials','sector_consumer_defensive','sector_technology',
             'sector_communication_services','sector_financial_services','sector_utilities','sector_industrials','sector_energy',
             'sector_healthcare','cashPosition','stockPosition','bondPosition','otherPosition','preferredPosition','convertiblePosition']
for col in fill_zero:
    etf_data_small[col] = etf_data_small[col].fillna(0)

In [367]:
# one row doesnt have beta, avg_monthly_return, RSI --> drop this row (Lyxor ETF)
etf_data_small = etf_data_small.dropna(subset=['beta'])

# 33 rows dont have var, volatility --> drop these rows (mostly Lyxor ETF)
etf_data_small = etf_data_small.dropna(subset=['Var'])

etf_data_small

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,sector_industrials,sector_energy,sector_healthcare,Var,drawdown,avg_sharpe_ratio,beta,avg_monthly_return,volatility,RSI
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0.1519,0.1534,0.0214,-0.012182,-0.224168,-0.943643,0.0,0.016163,0.009947,53.540314
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0.1520,0.1535,0.0214,-0.011809,-0.356065,-0.803378,0.0,0.013677,0.010545,53.611734
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,0.0000,0.0000,0.0000,-0.013404,-0.200601,-0.647148,0.0,0.030004,0.013205,65.807441
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0.3475,0.1443,0.0511,-0.013861,-0.423651,-0.784502,0.0,0.005198,0.011083,46.335950
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0.1226,0.1313,0.0000,-0.014308,-0.484438,-0.656542,0.0,0.010611,0.012936,46.086928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0.2787,0.0488,0.0344,-0.013612,-0.388548,-0.866023,0.0,0.003045,0.010576,43.097363
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0.2787,0.0488,0.0344,-0.012885,-0.383876,-0.866300,0.0,0.007272,0.010376,46.422888
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0.1596,0.0000,0.2001,-0.010444,-0.315128,-0.882943,0.0,0.011188,0.010002,50.094538
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0.1596,0.0000,0.2001,-0.009910,-0.309434,-0.890636,0.0,0.013374,0.009727,50.403902


In [368]:
# one-hot encode the labels

if 'labels' in etf_data_small.columns:
    etf_data_small['labels'] = etf_data_small['labels'].apply(clean_and_parse)
else:
    print("The 'labels' column does not exist in the DataFrame. Current columns are:", etf_data.columns)

unique_items = set([item for sublist in etf_data_small['labels'] for item in sublist])

new_columns = []
for item in unique_items: 
    new_column = etf_data_small['labels'].apply(lambda x: 1 if item in x else 0)
    new_column.name = item  
    new_columns.append(new_column)

etf_data_small = pd.concat([etf_data_small] + new_columns, axis=1)

etf_data_small

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,All-World,Change,Neutral,Equity,Convertible,3-5,Economy,1-3Y,30-15,Bank
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0,0,0,1,0,0,0,0,0,0
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0,0,0,1,0,0,0,0,0,0
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,0,0,0,1,0,0,0,0,0,0
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0,0,0,1,0,0,0,0,0,0
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0,0,0,1,0,0,0,0,0,0
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0,0,0,1,0,0,0,0,0,0
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0,0,0,1,0,0,0,0,0,0
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0,0,0,1,0,0,0,0,0,0


In [369]:
# drop labels column
etf_data_small = etf_data_small.drop(columns = 'labels')

In [370]:
# confirm data types

str_cols = ['isin', 'wkn', 'name', 'fundProvider', 'legalStructure', 'fundCurrency', 'inceptionDate', 'ticker', 'domicileCountry']

for column in etf_data_small.columns:
    if column in str_cols:
        etf_data_small[column] = etf_data_small[column].astype(str)
    else:
       etf_data_small[column] = pd.to_numeric(etf_data_small[column], errors='coerce')


In [371]:
etf_data_small

,isin,wkn,name,fundProvider,legalStructure,quote,ter,fundCurrency,inceptionDate,ticker,...,All-World,Change,Neutral,Equity,Convertible,3-5,Economy,1-3Y,30-15,Bank
0,IE00BMTX2B82,A2P9XA,iShares AEX UCITS ETF EUR (Acc),iShares,ETF,6.68,0.0030,EUR,2020-07-29,AYE7,...,0,0,0,1,0,0,0,0,0,0
1,NL0009272749,A1JN2C,VanEck AEX UCITS ETF,VanEck,ETF,72.34,0.0030,EUR,2009-12-14,2TCA,...,0,0,0,1,0,0,0,0,0,0
2,IE00BKPTXQ89,A2P4PH,HANetf Alerian Midstream Energy Dividend UCITS...,HANetf,ETF,10.70,0.0040,USD,2020-07-27,JMLP,...,0,0,0,1,0,0,0,0,0,0
3,NL0009272756,A1JPFU,VanEck AMX UCITS ETF,VanEck,ETF,92.64,0.0035,EUR,2009-12-14,2TCE,...,0,0,0,1,0,0,0,0,0,0
4,DE000A0D8Q23,A0D8Q2,iShares ATX UCITS ETF (DE),iShares,ETF,34.38,0.0032,EUR,2005-08-04,EXXX,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,IE00BQZJC527,A12HUU,WisdomTree Europe SmallCap Dividend UCITS ETF,WisdomTree,ETF,17.77,0.0038,EUR,2014-10-21,WTES,...,0,0,0,1,0,0,0,0,0,0
1017,IE00BDF16114,A2ARXA,WisdomTree Europe SmallCap Dividend UCITS ETF Acc,WisdomTree,ETF,17.50,0.0038,EUR,2016-11-03,WTD7,...,0,0,0,1,0,0,0,0,0,0
1018,IE00BZ56RN96,A2AG1D,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,25.98,0.0038,USD,2016-11-02,WTEQ,...,0,0,0,1,0,0,0,0,0,0
1019,IE00BZ56SW52,A2AG1E,WisdomTree Global Quality Dividend Growth UCIT...,WisdomTree,ETF,29.54,0.0038,USD,2016-06-03,WTEM,...,0,0,0,1,0,0,0,0,0,0
